In [ ]:
import pandas as pd
import seaborn as sns
from datetime import date
from sqlalchemy import create_engine

import os
import configparser

%matplotlib inline
sns.set_context("poster")

In [ ]:

config = configparser.ConfigParser()
config.read(os.path.join(os.path.expanduser("~"), '.config', 'me.ini'))

engine = create_engine(config['db']['url'])
aforro = pd.read_sql('aforro', engine)
monthly = pd.DataFrame(aforro.groupby('value_date')['value'].sum())


# get invested values
movements = pd.read_sql('account_movements', engine)
subs = movements[movements.description.str.contains('Subscrição C[AT]')]

invested = {
    'CAF Série B': subs[subs.account_id.isin([20, 21, 22, 23])].ammount.sum(),
    'CAF Série C': subs[subs.account_id.isin([24, 25])].ammount.sum(),
    'CT CTPM': subs[subs.account_id.isin([26, 27, 28])].ammount.sum(),
}


In [ ]:
monthly = aforro.pivot_table(index='value_date', columns='product', values='value', aggfunc=sum)

# winnings per product
for prod in invested.keys():
    monthly['Winnings ' + prod] = monthly[prod] - invested[prod]
monthly.fillna(0, inplace=True)

# total winnings
monthly['total winnings'] = monthly['Winnings CAF Série B'] + monthly['Winnings CAF Série C'] + monthly['Winnings CT CTPM']

# percentage. First get the total winnings, and then calculate percentages.
for prod in invested.keys():
    monthly['Perc Winnings ' + prod] = monthly['Winnings ' + prod] / monthly['total winnings']

In [ ]:
monthly

In [ ]:
monthly['total winnings'].plot(kind='bar')

In [ ]:
monthly[['Winnings CAF Série B', 'Winnings CAF Série C', 'Winnings CT CTPM']].plot(kind='bar', stacked=True)

In [ ]:
monthly[['Winnings CAF Série B', 'Winnings CAF Série C', 'Winnings CT CTPM']].plot(kind='area', stacked=True)

In [ ]:
monthly[monthly.index > '2014-09-01'][['Perc Winnings CAF Série B', 'Perc Winnings CAF Série C', 'Perc Winnings CT CTPM']].plot(kind='area', stacked=True)